In [132]:
import pandas as pd
import shap
from sklearn.discriminant_analysis import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim

In [133]:
# 加载 CSV 文件
data = pd.read_csv("../data/water/整理好的csv/杭州202101-202112/东苕溪202101-202112.csv", encoding="gb2312")

In [134]:
# 处理缺失值
data.fillna(method='ffill', inplace=True)

# 删除重复行
data.drop_duplicates(inplace=True)

C:\Users\admin\AppData\Local\Temp\ipykernel_14116\3374461071.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)


In [135]:
# 选择数值列（水温, pH, 溶解氧, 高锰酸钾, 氨氮, 总磷, 总氮, 电导率, 浊度）

numeric_columns = ['水温', 'pH', '溶解氧', '高锰酸钾', '氨氮', '总磷', '总氮', '电导率', '浊度']
data_numeric = data[numeric_columns]

# 数据标准化处理
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_numeric)

# 转换为Pandas DataFrame格式
data_scaled_df = pd.DataFrame(data_scaled, columns=numeric_columns)

In [136]:
# 定义VAE模型
class VAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(VAE, self).__init__()
        # 编码器
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc2_logvar = nn.Linear(hidden_dim, latent_dim)
        # 解码器
        self.fc3 = nn.Linear(latent_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, input_dim)

    def encode(self, x):
        h1 = torch.relu(self.fc1(x))
        return self.fc2_mu(h1), self.fc2_logvar(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h3 = torch.relu(self.fc3(z))
        return self.fc4(h3)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

# 定义损失函数
def loss_function(recon_x, x, mu, logvar):
    # 使用MSE作为重构损失
    MSE = nn.functional.mse_loss(recon_x, x, reduction='sum')
    # KL散度损失
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return MSE + KLD

# 训练VAE模型
def train_vae(data, input_dim, hidden_dim=64, latent_dim=16, epochs=100, batch_size=32, learning_rate=1e-3):
    model = VAE(input_dim, hidden_dim, latent_dim)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    data_tensor = torch.FloatTensor(data)

    dataset = torch.utils.data.TensorDataset(data_tensor)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for batch_idx, (inputs,) in enumerate(dataloader):
            optimizer.zero_grad()
            recon_batch, mu, logvar = model(inputs)
            loss = loss_function(recon_batch, inputs, mu, logvar)
            loss.backward()
            train_loss += loss.item()
            optimizer.step()

        print(f'Epoch {epoch+1}, Loss: {train_loss / len(dataloader.dataset)}')

    return model

# 使用VAE模型进行异常检测
def detect_anomalies(model, data, threshold=1.8):
    data_tensor = torch.FloatTensor(data)
    model.eval()
    with torch.no_grad():
        recon_data, _, _ = model(data_tensor)
        reconstruction_error = torch.mean((recon_data - data_tensor) ** 2, dim=1)
        anomalies = reconstruction_error > threshold
    return anomalies.numpy(), reconstruction_error.numpy()

# 使用SHAP解释每个特征对异常的贡献
def explain_anomalies(model, data, sample_data):
    # 定义解释器函数
    def model_predict(inputs):
        model.eval()
        with torch.no_grad():
            recon_data, _, _ = model(torch.FloatTensor(inputs))
            reconstruction_error = torch.mean((recon_data - torch.FloatTensor(inputs)) ** 2, dim=1)
        return reconstruction_error.numpy()

    # 创建一个KernelExplainer来计算SHAP值
    explainer = shap.KernelExplainer(model_predict, data)
    
    # 计算SHAP值
    shap_values = explainer.shap_values(sample_data)

    return shap_values

# 设置参数并训练VAE模型
input_dim = data_scaled_df.shape[1]
vae_model = train_vae(data_scaled_df.values, input_dim, epochs=100)

# 检测异常
anomalies, recon_error = detect_anomalies(vae_model, data_scaled_df.values)

# 将检测结果显示给用户
anomalies_df = data.copy()
anomalies_df['Reconstruction Error'] = recon_error
anomalies_df['Anomaly'] = anomalies

# 筛选出异常数据点
anomalous_data = data_scaled_df.values[anomalies]

# 解释异常值的SHAP值
shap_values = explain_anomalies(vae_model, data_scaled_df.values, anomalous_data)

# 创建一个DataFrame来保存SHAP值
shap_df = pd.DataFrame(shap_values, columns=data_scaled_df.columns)

# 将SHAP值添加到异常数据中
anomalies_df = anomalies_df[anomalies_df['Anomaly'] == True]
for column in shap_df.columns:
    anomalies_df[f'SHAP_{column}'] = shap_df[column].values

anomalies_df

anomalies_df.to_excel('test2.xlsx',sheet_name='Sheet1',index=False)

Epoch 1, Loss: 8.879900251969566
Epoch 2, Loss: 7.60110879299131
Epoch 3, Loss: 7.0894327962781265
Epoch 4, Loss: 6.692410211406281
Epoch 5, Loss: 6.5824528024265705
Epoch 6, Loss: 6.453636713110103
Epoch 7, Loss: 6.228686142266497
Epoch 8, Loss: 6.137854672449122
Epoch 9, Loss: 6.03765026740759
Epoch 10, Loss: 5.859190944590975
Epoch 11, Loss: 5.897212859701158
Epoch 12, Loss: 5.893812659016417
Epoch 13, Loss: 5.814452563400089
Epoch 14, Loss: 5.797815745569526
Epoch 15, Loss: 5.770141219196604
Epoch 16, Loss: 5.684019215702544
Epoch 17, Loss: 5.665281697559282
Epoch 18, Loss: 5.57834084554865
Epoch 19, Loss: 5.580752146458757
Epoch 20, Loss: 5.5351528744260685
Epoch 21, Loss: 5.610554694382587
Epoch 22, Loss: 5.590305337330841
Epoch 23, Loss: 5.503229414567223
Epoch 24, Loss: 5.509611621160664
Epoch 25, Loss: 5.485653022162692
Epoch 26, Loss: 5.4592162380950375
Epoch 27, Loss: 5.438826720044308
Epoch 28, Loss: 5.369049305269844
Epoch 29, Loss: 5.423484772373758
Epoch 30, Loss: 5.3652

Using 2554 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


Epoch 100, Loss: 4.9874785385191585


  0%|          | 0/11 [00:00<?, ?it/s]

In [137]:
# 加载 CSV 文件
data = pd.read_csv("../data/water/整理好的csv/杭州202101-202112/京杭运河202101-202112.csv", encoding="gb2312")
# 处理缺失值
data.fillna(method='ffill', inplace=True)

# 删除重复行
data.drop_duplicates(inplace=True)
# 选择数值列（水温, pH, 溶解氧, 高锰酸钾, 氨氮, 总磷, 总氮, 电导率, 浊度）
numeric_columns = ['水温', 'pH', '溶解氧', '高锰酸钾', '氨氮', '总磷', '总氮', '电导率', '浊度']
data_numeric = data[numeric_columns]

# 数据标准化处理
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_numeric)

# 转换为Pandas DataFrame格式
data_scaled_df = pd.DataFrame(data_scaled, columns=numeric_columns)

C:\Users\admin\AppData\Local\Temp\ipykernel_14116\368478832.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)


In [138]:
# 定义一个函数用于加载新数据并进行异常检测
def detect_anomalies_on_new_data(model, new_data, threshold=8):
    # 将新数据转化为Tensor
    new_data_tensor = torch.FloatTensor(new_data)
    
    # 使用训练好的模型进行推理
    model.eval()
    with torch.no_grad():
        recon_data, _, _ = model(new_data_tensor)
        reconstruction_error = torch.mean((recon_data - new_data_tensor) ** 2, dim=1)
        anomalies = reconstruction_error > threshold
    
    # 返回检测结果和重构误差
    return anomalies.numpy(), reconstruction_error.numpy()

# 使用SHAP解释每个特征对新数据异常的贡献
def explain_anomalies_on_new_data(model, new_data, sample_data):
    # 定义解释器函数
    def model_predict(inputs):
        model.eval()
        with torch.no_grad():
            recon_data, _, _ = model(torch.FloatTensor(inputs))
            reconstruction_error = torch.mean((recon_data - torch.FloatTensor(inputs)) ** 2, dim=1)
        return reconstruction_error.numpy()

    # 创建一个KernelExplainer来计算SHAP值
    explainer = shap.KernelExplainer(model_predict, new_data)
    
    # 计算SHAP值
    shap_values = explainer.shap_values(sample_data)

    return shap_values

# 加载新的数据集（确保已经预处理，且维度与原始数据集相同）
new_data_scaled_df = data_scaled_df  # 你需要加载和预处理新的数据

# 使用训练好的模型检测新数据集中的异常
anomalies_new, recon_error_new = detect_anomalies_on_new_data(vae_model, new_data_scaled_df.values)

# 将结果显示给用户
new_anomalies_df = new_data_scaled_df.copy()
new_anomalies_df['Reconstruction Error'] = recon_error_new
new_anomalies_df['Anomaly'] = anomalies_new

# 显示异常的行
data[new_anomalies_df['Anomaly'] == True]

# 筛选出新数据中的异常点
anomalous_new_data = new_data_scaled_df[anomalies_new]

# 解释新数据中异常点的SHAP值
shap_values_new = explain_anomalies_on_new_data(vae_model, new_data_scaled_df.values, anomalous_new_data.values)

# 创建一个DataFrame来保存新数据的SHAP值
shap_new_df = pd.DataFrame(shap_values_new, columns=new_data_scaled_df.columns)

# 将SHAP值添加到新数据的异常检测结果中
anomalous_new_df = new_anomalies_df[new_anomalies_df['Anomaly'] == True].copy()
for column in shap_new_df.columns:
    anomalous_new_df[f'SHAP_{column}'] = shap_new_df[column].values

anomalous_new_df

Using 4005 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


  0%|          | 0/13 [00:00<?, ?it/s]

,水温,pH,溶解氧,高锰酸钾,氨氮,总磷,总氮,电导率,浊度,Reconstruction Error,Anomaly,SHAP_水温,SHAP_pH,SHAP_溶解氧,SHAP_高锰酸钾,SHAP_氨氮,SHAP_总磷,SHAP_总氮,SHAP_电导率,SHAP_浊度
3,58.017171,-0.016082,-0.022070,-0.016843,-0.018440,-0.020728,-0.019263,-0.026910,-0.016236,344.436523,True,606.658768,-28.104217,-30.035960,-30.521546,-29.654380,-29.703611,-30.381305,-30.798143,-31.274724
5,-0.027265,-0.016054,47.544491,-0.016859,-0.018440,-0.020721,-0.019265,-0.026766,-0.016096,193.439133,True,0.260123,0.395953,196.853020,0.372964,0.398165,0.381221,0.330143,0.355816,0.420775
8,-0.027614,-0.016077,-0.022088,-0.016838,-0.018373,58.486352,-0.019249,-0.025366,-0.016139,135.718750,True,0.845734,1.049524,0.935574,1.013111,0.990447,142.263431,0.899820,0.937429,1.061030
9,-0.027324,-0.016054,-0.022110,-0.016840,-0.018385,-0.020716,15.117260,-0.025402,-0.016117,17.213760,True,-0.054585,0.076259,-0.002663,0.066020,0.088322,0.061145,15.354532,0.010312,0.095642
10,-0.027265,-0.016054,-0.022017,-0.016846,-0.018379,-0.020715,-0.019251,63.267827,-0.016056,348.434906,True,-2.353653,-2.213099,-2.264317,-2.239530,-2.165639,-2.285252,-2.290672,364.566123,-2.276019
11,-0.027207,-0.016049,-0.022095,-0.016859,-0.018371,-0.020714,-0.019257,-0.025433,63.277155,39.272381,True,-0.404361,-0.300780,-0.350787,-0.320234,-0.331392,-0.255614,-0.332187,-0.371638,35.066589
13,-0.027324,-0.016063,-0.022114,-0.016797,-0.018367,-0.020713,61.441063,-0.025038,-0.016132,349.617371,True,-0.748404,-0.607088,-0.699223,-0.632279,-0.585781,-0.665993,351.070696,-0.687588,-0.585843
14,-0.028252,-0.016002,-0.021730,-0.016177,-0.018245,24.137962,-0.019172,-0.018084,-0.015420,20.485832,True,0.003251,0.149015,0.070930,0.168439,0.173270,20.155835,0.067821,0.110394,0.177674
15,-0.027440,-0.016072,-0.022136,-0.016824,62.324160,-0.020709,-0.019252,-0.024684,-0.016088,54.209415,True,-0.448686,-0.219637,-0.194723,-0.147083,52.213175,-0.092689,-0.159831,-0.244143,-0.168185
16,-0.027846,-0.015992,-0.021764,63.189568,-0.018244,-0.020652,-0.019079,-0.018110,-0.015373,197.514160,True,-0.327004,-0.195207,-0.253513,200.805018,-0.163666,-0.181012,-0.270357,-0.262507,-0.188368


In [140]:
error=data[new_anomalies_df['Anomaly'] == True]

error.to_excel('test1.xlsx',sheet_name='Sheet1',index=False)

error

,省份,城市,河流,流域,断面名称,监测时间,水质类别,水温,pH,溶解氧,高锰酸钾,氨氮,总磷,总氮,电导率,浊度
3,浙江省,杭州市,京杭运河,太湖流域,顾家桥,2021/1/1 12:00,Ⅱ,100010.0,7.59,9.80,1.85,0.097,0.026,1.17,182.7,29.3
5,浙江省,杭州市,京杭运河,太湖流域,顾家桥,2021/1/1 20:00,Ⅱ,9.7,7.65,516431.00,1.79,0.098,0.033,1.15,185.5,56.6
8,浙江省,杭州市,京杭运河,太湖流域,顾家桥,2021/1/2 8:00,Ⅱ,9.1,7.60,9.60,1.87,0.230,56752.000,1.29,212.8,48.2
9,浙江省,杭州市,京杭运河,太湖流域,顾家桥,2021/1/2 12:00,Ⅱ,9.6,7.65,9.37,1.86,0.207,0.038,134134.00,212.1,52.4
10,浙江省,杭州市,京杭运河,太湖流域,顾家桥,2021/1/2 16:00,Ⅱ,9.7,7.65,10.37,1.84,0.218,0.039,1.28,1234454.0,64.3
11,浙江省,杭州市,京杭运河,太湖流域,顾家桥,2021/1/2 20:00,Ⅱ,9.8,7.66,9.53,1.79,0.235,0.040,1.22,211.5,12341234.0
13,浙江省,杭州市,京杭运河,太湖流域,顾家桥,2021/1/3 4:00,Ⅱ,9.6,7.63,9.32,2.02,0.243,0.041,544634.00,219.2,49.6
14,浙江省,杭州市,京杭运河,太湖流域,五杭运河大桥,2021/1/3 8:00,Ⅲ,8.0,7.76,13.49,4.32,0.482,23434.000,1.98,354.8,188.3
15,浙江省,杭州市,京杭运河,太湖流域,顾家桥,2021/1/3 8:00,Ⅱ,9.4,7.61,9.08,1.92,123123.000,0.045,1.27,226.1,58.0
16,浙江省,杭州市,京杭运河,太湖流域,五杭运河大桥,2021/1/3 12:00,Ⅲ,8.7,7.78,13.12,234326.00,0.484,0.100,2.80,354.3,197.6


In [149]:
# 获取异常数据的行号
anomalous_indices = new_anomalies_df.index[new_anomalies_df['Anomaly'] == True].tolist()

# 找到每个异常数据异常的特征列
anomalous_coords = []

for i, row in shap_new_df.iterrows():
    # 对每个异常点的SHAP值进行分析，找出影响最大的特征
    for col in shap_new_df.columns:
        if abs(row[col]) > 40:  # 设置一个阈值，比如0.1，表示该特征对异常有较大的贡献
            # 获取异常行号和原始数据值
            index = anomalous_indices[i]
            original_value = error.loc[index, col]  #anomalous_new_data;data_scaled_df;new_data_scaled_df
            shap_value = row[col]
            # 记录行号、特征、原始数值和SHAP值
            anomalous_coords.append((index, col, original_value, shap_value))

# 打印出异常数据的坐标及其特征贡献
for coord in anomalous_coords:
    print(f"行号: {coord[0]}, 特征: {coord[1]}, 原始数值: {coord[2]}, SHAP值: {coord[3]}")

# 如果你想保存异常数据的坐标和特征到Excel文件
# 将结果保存到DataFrame
anomalous_coords_df = pd.DataFrame(anomalous_coords, columns=['行号', '特征', '原始数值', 'SHAP值'])

# 保存到Excel
anomalous_coords_df.to_excel('test3.xlsx', sheet_name='异常数据坐标', index=False)

行号: 3, 特征: 水温, 原始数值: 100010.0, SHAP值: 606.6587676545695
行号: 5, 特征: 溶解氧, 原始数值: 516431.0, SHAP值: 196.85302045153102
行号: 8, 特征: 总磷, 原始数值: 56752.0, SHAP值: 142.2634309885723
行号: 10, 特征: 电导率, 原始数值: 1234454.0, SHAP值: 364.56612254730516
行号: 13, 特征: 总氮, 原始数值: 544634.0, SHAP值: 351.07069553905166
行号: 15, 特征: 氨氮, 原始数值: 123123.0, SHAP值: 52.213174616774566
行号: 16, 特征: 高锰酸钾, 原始数值: 234326.0, SHAP值: 200.80501837452474
行号: 17, 特征: 溶解氧, 原始数值: 453452.0, SHAP值: 150.70313098391918
行号: 18, 特征: pH, 原始数值: 134445.0, SHAP值: 104.44896143077095
行号: 19, 特征: 水温, 原始数值: 43546.0, SHAP值: 64.17677617990292
